In [11]:
""" stock universe construction """
import pandas as pd 
import numpy as np 
# read stock_header.csv; contains permno, shrcd (share code)
sh = pd.read_csv('/Users/codywan/Data/WRDS Data/stock_header.csv', header=0, keep_default_na=False, index_col=0)
# read crspa_msf.csv; contains monthly stock price, shares outstanding, cumulative factors for adjusting price and volume etc. 
msf = pd.read_csv('/Users/codywan/Data/WRDS Data/crspa_msf.csv', header=0, keep_default_na=False, index_col=0)

# remove rows with no end-of-month price or shares outstanding
msf['prc'] = msf['prc'].replace('', np.nan).astype('float64').abs()
msf['shrout'] = msf['shrout'].replace('', np.nan).astype('float64').abs()
msf['cfacpr'] = msf['cfacpr'].replace('', np.nan).astype('float64').abs()
msf['cfacshr'] = msf['cfacshr'].replace('', np.nan).astype('float64').abs()
msf = msf.dropna(subset=['prc', 'shrout', 'cfacpr', 'cfacshr'])

In [12]:
# print how many unique permno there are
permno_list = msf['permno'].unique()
print(f"number of unique permno:\n{len(permno_list)}")
# only keep permno/stocks with a share code of 10 or 11 (i.e. only keep U.S. common stocks)
permno_shrcd_mapping = sh[['permno', 'hshrcd']].set_index('permno')['hshrcd'].to_dict()
permno_list = [permno for permno in permno_list if (permno_shrcd_mapping[permno] == 10.0) or (permno_shrcd_mapping[permno] == 11.0)]
# print how many permno there're after removing non-10/11 share code stock
print(f"removing non U.S. common stock:\n{len(permno_list)}")

number of unique permno:
32985
removing non U.S. common stock:
24781


In [13]:
# remove stocks listed outside of time interval of interest and ones without at least 24 months of price history
# reason for starting at 1996 (configurable):
# "... since May of 1996, all reporting companies have been required to file electronically...." 
# https://help.edgar-online.com/edgar/history.asp?site=pro 
MIN_PRICE_HISTORY = 24 # months
T0 = pd.to_datetime('01/05/1996')
T1 = pd.to_datetime('01/01/2020')
# filtering
temp = list()
for permno in permno_list:
    dates = pd.to_datetime(msf[msf['permno'] == permno]['date'].values)
    if dates[0] >= T1 or dates[-1] < T0: # filtering by time interval
        continue
    if ((dates>=T0) & (dates<=T1)).sum() < MIN_PRICE_HISTORY: # filtering by # of price history
        continue
    temp.append(permno)
permno_list = temp
print(f"removing permno without at least {MIN_PRICE_HISTORY} months of return history from {T0} to {T1} \n{len(permno_list)}")

removing permno without at least 24 months of return history from 1996-01-05 00:00:00 to 2020-01-01 00:00:00 
11623


In [14]:
# remove stocks that fall below a certain threshold of market cap

MIN_MARKET_CAP = 500000 # thousand
if_all = True # True: has to be above market cap threshold at every price point; False: has to beabove threshold at at least one price point

temp = list()
for permno in permno_list:
    permno_series = msf[msf['permno'] == permno]
    # computes market cap
    market_cap_series = permno_series['prc'] * permno_series['shrout'] / permno_series['cfacpr'] * permno_series['cfacshr']
    # filtering
    if if_all and all(market_cap_series >= MIN_MARKET_CAP):
        temp.append(permno)
    elif (not if_all) and any(market_cap_series >= MIN_MARKET_CAP):
        temp.append(permno)
permno_list = temp
print(f"removing permno that falls under {MIN_MARKET_CAP:,} thousand market cap \n{len(permno_list)}")

removing permno that falls under 500,000 thousand market cap 
775


In [15]:
# more filtering can be done as needed

In [16]:
# save permno to a csv file
import csv
with open(f"data/filtered_permno_list_{T0.strftime('%Y%m%d')}_{T1.strftime('%Y%m%d')}.csv", 'wt') as f:
    wr = csv.writer(f)
    wr.writerow(permno_list)

In [17]:
""" link permno to SEC edgar identifier """
import csv
import pandas as pd
import numpy as np 

with open(f"data/filtered_permno_list_{T0.strftime('%Y%m%d')}_{T1.strftime('%Y%m%d')}.csv") as f:
    reader = csv.reader(f)
    permno_list = list(reader)[0]
permno_list = [int(float(p)) for p in permno_list]
print(f"total number of filtered permno:\n{len(permno_list)}")

total number of filtered permno:
775


In [18]:
# mapping of gvkey, permno, permco, ticker, cik etc.
link = pd.read_csv('/Users/codywan/Data/WRDS Data/crspa_ccmlinktable.csv', header=0, keep_default_na=False).replace("", np.NaN).dropna(subset=['LPERMNO', 'cik'])
link[['LPERMNO', 'LPERMCO', 'GVKEY', 'cik']] = link[['LPERMNO', 'LPERMCO', 'GVKEY', 'cik']].astype('int')

# permno to permco (int)
permno_permco_mapping = link[['LPERMNO', 'LPERMCO']].set_index('LPERMNO')['LPERMCO'].to_dict() # all permno to permco mapping
permno_permco_mapping = {permno: permno_permco_mapping.get(permno, np.NaN) for permno in permno_list} # filter out permno not in permno_list
print(f"{sum([1 for permno in permno_permco_mapping if not np.isnan(permno_permco_mapping[permno])])}/{len(permno_permco_mapping)} permno mapped to non-nan permco, {len(set(list(permno_permco_mapping.values())))} unique permco values")
# permno to gvkey (int)
permno_gvkey_mapping = link[['LPERMNO', 'GVKEY']].set_index('LPERMNO')['GVKEY'].to_dict() # all permno to gvkey mapping
permno_gvkey_mapping = {permno: permno_gvkey_mapping.get(permno, np.NaN) for permno in permno_list} # filter out permno not in permno_list
print(f"{sum([1 for permno in permno_gvkey_mapping if not np.isnan(permno_gvkey_mapping[permno])])}/{len(permno_gvkey_mapping)} permno mapped to non-nan gvkey, {len(set(list(permno_gvkey_mapping.values())))} unique gvkey values")
# permno to cusip (char)
permno_cusip_mapping = link[['LPERMNO', 'cusip']].set_index('LPERMNO')['cusip'].to_dict() # all permno to cusip mapping
permno_cusip_mapping = {permno: permno_cusip_mapping.get(permno, "") for permno in permno_list} # filter out permno not in permno_list
print(f"{sum([1 for permno in permno_cusip_mapping if permno_cusip_mapping[permno] != ''])}/{len(permno_cusip_mapping)} permno mapped to non-nan cusip, {len(set(list(permno_cusip_mapping.values())))} unique cusip values")
# permno to cik (int)
permno_cik_mapping = link[['LPERMNO', 'cik']].set_index('LPERMNO')['cik'].to_dict() # all permno to cik mapping
permno_cik_mapping = {permno: permno_cik_mapping.get(permno, np.NaN) for permno in permno_list} # filter out permno not in permno_list
print(f"{sum([1 for permno in permno_cik_mapping if not np.isnan(permno_cik_mapping[permno])])}/{len(permno_cik_mapping)} permno mapped to non-nan cik, {len(set(list(permno_cik_mapping.values())))} unique cik values")

757/775 permno mapped to non-nan permco, 741 unique permco values
757/775 permno mapped to non-nan gvkey, 729 unique gvkey values
757/775 permno mapped to non-nan cusip, 729 unique cusip values
757/775 permno mapped to non-nan cik, 729 unique cik values


In [19]:
# save permno to cik mapping to csv
df = pd.DataFrame.from_dict(permno_cik_mapping, orient='index', columns=['cik'])
# time format is YYYYMMDD, to be consistent with the one used for making web request in following steps
df.to_csv(f"data/permno_cik_mapping_{T0.strftime('%Y%m%d')}_{T1.strftime('%Y%m%d')}_MinMK_{MIN_MARKET_CAP}.csv")

""" end of identifier_preprocessing """

' end of identifier_preprocessing '

In [ ]:
""" a few note-worthy cases in stock identifier mapping """
# htsymbols are not unique
# EGAS were used by GAS Natural Inc and Energy Search Inc for some overlapping period?
# one is NYSE: EGAS, the other one is NASDAQ: EGAS
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(sh[sh['htsymbol']=='EGAS'])

In [ ]:
# no record for some permno in linking file
# e.g. permno: 16267, Dell Technologies Inc
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(link[link['LPERMNO']==16267])

In [ ]:
# negative cik for permno 13757, 13758, etc
# verify if cik is negative link
link = pd.read_csv('/Users/codywan/Data/WRDS Data/crspa_ccmlinktable.csv', header=0, keep_default_na=False).replace("", np.NaN)
link[link['LPERMNO']==75298]
# turned out it was np.nan being being converted to int and parsed as some negative number by pandas